# Azure AI Search

[Azure AI Search](https://learn.microsoft.com/azure/search/search-what-is-azure-search) (formerly known as `Azure Search` and `Azure Cognitive Search`) is a cloud search service that gives developers infrastructure, APIs, and tools for information retrieval of vector, keyword, and hybrid queries at scale.

You'll need to install `langchain-community` with `pip install -qU langchain-community` to use this integration

## Install Azure AI Search SDK

Use azure-search-documents package version 11.4.0 or later.

https://github.com/langchain-ai/langchain/blob/master/templates/rag-azure-search/README.md
https://github.com/langchain-ai/langchain/blob/master/docs/docs/integrations/vectorstores/azuresearch.ipynb

In [ ]:
%pip install --upgrade --quiet  azure-search-documents
%pip install --upgrade --quiet  azure-identity
%pip install --upgrade --quiet  langchain-community
%pip install --upgrade --quiet  langchain_openai

## Import required libraries

`OpenAIEmbeddings` is assumed, but if you're using Azure OpenAI, import `AzureOpenAIEmbeddings` instead.

In [1]:
import os

from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings

## Configure OpenAI settings
Set variables for your OpenAI provider. You need either an [OpenAI account](https://platform.openai.com/docs/quickstart?context=python) or an [Azure OpenAI account](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource) to generate the embeddings. 

In [2]:
from dotenv import load_dotenv
# Get configuration settings
load_dotenv()
# use an Azure OpenAI account with a deployment of an embedding model
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

In [9]:
print(azure_endpoint)
print(azure_deployment)
print(azure_openai_api_version)

https://demooai01.openai.azure.com/
text-embedding-3-large
2024-09-01-preview


## Configure vector store settings

You need an [Azure subscription](https://azure.microsoft.com/en-us/free/search) and [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal) to use this vector store integration. No-cost versions are available for small and limited workloads.
 
Set variables for your Azure AI Search URL and admin API key. You can get these variables from the [Azure portal](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.Search%2FsearchServices).

In [4]:
vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_KEY")

## Create embeddings and vector store instances
 
Create instances of the OpenAIEmbeddings and AzureSearch classes. When you complete this step, you should have an empty search index on your Azure AI Search resource. The integration module provides a default schema.

In [5]:
# Use AzureOpenAIEmbeddings with an Azure account
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)

## Create vector store instance
 
Create instance of the AzureSearch class using the embeddings from above

In [6]:
index_name: str = "langchain-vector-index"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [8]:
print(embeddings.embed_query)

<bound method OpenAIEmbeddings.embed_query of AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000201744118B0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000020174CA96A0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-3-large', openai_api_version='2024-09-01-preview', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://demooai01.openai.azure.com/', azure_ad_token=None, azure_ad_token_provi

In [10]:
# Specify additional properties for the Azure client such as the following https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/core/azure-core/README.md#configurations
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    # Configure max retries for the Azure client
    additional_search_client_options={"retry_total": 4},
)

## Insert text and embeddings into vector store
 
This step loads, chunks, and vectorizes the sample document, and then indexes the content into a search index on Azure AI Search.

In [24]:
#https://github.com/langchain-ai/langchain/issues/11313
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import CharacterTextSplitter


loader = CSVLoader("./data/WineDataset.csv",  encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# You might experience RequestEntityTooLargeError, to address this you can modify the batch size and chunk_size.
#vector_store.add_documents(documents=docs)

batch_size = 100  # Set batch size, adjust according to Azure's limitations
for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    vector_store.add_documents(documents=batch)  # Add smaller batches


Created a chunk of size 670, which is longer than the specified 500
Created a chunk of size 625, which is longer than the specified 500
Created a chunk of size 610, which is longer than the specified 500
Created a chunk of size 536, which is longer than the specified 500
Created a chunk of size 705, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500
Created a chunk of size 618, which is longer than the specified 500
Created a chunk of size 656, which is longer than the specified 500
Created a chunk of size 605, which is longer than the specified 500
Created a chunk of size 730, which is longer than the specified 500
Created a chunk of size 624, which is longer than the specified 500
Created a chunk of size 509, which is longer than the specified 500
Created a chunk of size 615, which is longer than the specified 500
Created a chunk of size 1026, which is longer th

## Perform a vector similarity search
 
Execute a pure vector similarity search using the similarity_search() method:

In [50]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What are the most expensive wines that won an IWC award",
    k=3,
    search_type="similarity",
)
from pprint import pprint

pprint(docs)
#print(docs[0].page_content)

[Document(metadata={'id': 'Nzk3NzA3NGMtZDMyOS00N2NhLThjNjYtZDljNjI0ZmJhZWQ1', 'source': './data/WineDataset.csv', 'row': 1247}, page_content='In 2022, this wine won an IWC award for the 2011 vintage.\nPrice: £166.99 per bottle\nCapacity: 75CL\nGrape: Chardonnay\nSecondary Grape Varieties: \nClosure: Natural Cork\nCountry: France\nUnit: 9.4\nCharacteristics: Citrus Fruit, Almond, Biscuit, Bread\nPer bottle / case / each: per bottle\nType: White\nABV: ABV 12.50%\nRegion: \nStyle: Rich & Toasty\nVintage: 2012\nAppellation:'),
 Document(metadata={'id': 'M2ExM2YwYmMtNmE5My00Y2FhLWI5MjgtYTNiM2ViZTZlNjBj', 'source': './data/WineDataset.csv', 'row': 700}, page_content='In 2022, this wine won an IWC award for the 2020 vintage.\nPrice: £18.99 per bottle\nCapacity: 75CL\nGrape: Cabernet Sauvignon\nSecondary Grape Varieties: Petit Verdot, Malbec, Merlot\nClosure: Screwcap\nCountry: South Africa\nUnit: 10.9\nCharacteristics: Tobacco, Black Plum, Blackberry, Blackcurrant, Leather\nPer bottle / case 

## Perform a vector similarity search with relevance scores
 
Execute a pure vector similarity search using the similarity_search_with_relevance_scores() method. Queries that don't meet the threshold requirements are exluded.

In [54]:
docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query="Which red wines go well with red meat?",
    k=4,
    score_threshold=0.64,
)
from pprint import pprint

pprint(docs_and_scores)

[(Document(metadata={'id': 'NDY3YzgwNGQtODFhZC00NmQ0LWEyOGMtNDVlMzU3NjU0MWE3', 'source': './data/WineDataset.csv', 'row': 126}, page_content='Title: Beefsteak Club Malbec 2021/22, Mendoza\nDescription: Argentina is a paradise for steak and Malbec lovers – and no one does them better. This wine is a tribute to the dining clubs of the 18th and 19th centuries, which celebrated steak as a symbol of prosperity. The grapes come from high-altitude vineyards in some of Mendoza’s best sites, so you can expect delicious flavour in every glassful. It’s juicy and smooth with ripe black-fruit notes. Oak ageing gives it lush vanilla and gently spiced aromas – all balanced by a delicious freshness. It’s a real crowd pleaser and, naturally, incredible with steak.'),
  0.6618109),
 (Document(metadata={'id': 'ZjRiNjJjNTUtZjM3My00ODZmLTg2YTYtM2JkOGYzOTM2MWJi', 'source': './data/WineDataset.csv', 'row': 78}, page_content="Title: Peter Lehmann 'The Barossan' Shiraz 2019/20, Barossa\nDescription: With rolli

## Perform a hybrid search

Execute hybrid search using the search_type or hybrid_search() method. Vector and nonvector text fields are queried in parallel, results are merged, and top matches of the unified result set are returned.

In [51]:
# Perform a hybrid search using the search_type parameter
docs = vector_store.similarity_search(
    query="Which red wines go well with red meat?",
    k=3,
    search_type="hybrid",
)
print(docs[0].page_content)

Title: Santa Rita 'Medalla Real' Carménère 2019/21, Colchagua Valley
Description: Carménère may be Chile’s flagship red grape. But did you know that its history stretches back to Bordeaux? When the tiny phylloxera bugs ravaged Europe in the nineteenth century, Bordeaux’s Carménère vines were a casualty. It had been a key component of some of the greatest claret at the time – but the vine was only saved by cuttings which had been taken to Chile.  So it’s no wonder that Santa Rita use Carménère to make this fantastically structured Gran Reserva, which is styled like a classic Bordeaux. It’s a plush, velvety-smooth wine, packed with intense flavours of blueberry, plum and cassis. The firm tannins make it ideal with red meat.
Price: £13.99 per bottle
Capacity: 75CL
Grape: Carménère
Secondary Grape Varieties: 
Closure: Natural Cork
Country: Chile
Unit: 10.5
Characteristics: Vanilla, Black Plum, Blackberry, Blackcurrant, Blueberry, Chocolate, Earth, Leather, Smoke, Tobacco
Per bottle / case 

In [58]:
# Perform a hybrid search using the hybrid_search method
docs = vector_store.hybrid_search(
    query="Which red wines go well with red meat?", k=3
)
print(docs[0].page_content)

Title: Majestic Classics 12 Red Wine Case
Description: A varied selection of delicious red wines, perfect for gatherings or enjoying with hearty roasts. It ticks all the boxes and you'll see why. This includes a juicy Spanish Tempranillo, a peppery South African Shiraz, a rich Argentinian Malbec and a French Rhone.
Price: £90.00 per case
Capacity: Our
Grape: 
Secondary Grape Varieties: 
Closure: 
Country: 
Unit: 
Characteristics: 
Per bottle / case / each: per case
Type: Red
ABV: 
Region: 
Style: 
Vintage: 
Appellation:


## Custom schemas and queries

This section shows you how to replace the default schema with a custom schema.


### Create a new index with custom filterable fields 

This schema shows field definitions. It's the default schema, plus several new fields attributed as filterable. Because it's using the default vector configuration, you won't see vector configuration or vector profile overrides here. The name of the default vector profile is "myHnswProfile" and it's using a vector configuration of Hierarchical Navigable Small World (HNSW) for indexing and queries against the content_vector field.

There's no data for this schema in this step. When you execute the cell, you should get an empty index on Azure AI Search.

## 1. Hierarchical Navigable Small World (HNSW)
### Purpose: 
HNSW is an approximate nearest neighbor (ANN) algorithm. It’s designed to quickly find the nearest vectors (similar documents or search results) to a query vector, while maintaining good accuracy.
How It Works: HNSW uses a graph-based approach where data points (vectors) are stored as nodes in a multi-layer graph. When searching for the nearest neighbors, it navigates through these layers in a hierarchical manner. The higher layers of the graph contain fewer nodes but help in quickly narrowing down the search space, while lower layers contain more nodes for a more detailed search.
### Advantages:
Speed: HNSW is very fast for large datasets due to its approximate nature. It doesn't compare every vector in the dataset, instead navigating the graph structure to reach a result quickly.
Scalability: It scales well with large datasets, making it ideal for search indexes where speed is a priority.
Memory Efficient: It has optimized memory usage compared to brute-force methods.
### Disadvantages:
Approximate Results: Since it is an approximate algorithm, the search results may not always be the true nearest neighbors. However, the trade-off in accuracy is often minimal and acceptable for most applications.
## 2. Exhaustive KNN (K-Nearest Neighbor)
### Purpose: 
Exhaustive KNN is a brute-force nearest neighbor search algorithm. It computes the distance between the query vector and every other vector in the dataset, ensuring the most accurate nearest neighbors.
How It Works: For each query, exhaustive KNN compares the query vector to all vectors in the dataset and returns the k vectors that are closest to the query based on a similarity measure (like cosine similarity or Euclidean distance).
### Advantages:
Accuracy: This method guarantees the exact nearest neighbors, making it the most accurate method.
Simplicity: It’s straightforward in implementation, as it does not require building complex data structures like in HNSW.
### Disadvantages:
Speed: Exhaustive KNN is significantly slower than approximate methods, especially with large datasets, as it needs to compute distances for every vector in the dataset.
Scalability: It becomes inefficient as the dataset grows larger, making it less suitable for real-time applications where speed is critical.
Resource Intensive: Since it processes every vector in the dataset, it requires more computation and memory resources compared to HNSW.
## When to Use:
### HNSW:
Ideal for large datasets and applications where speed is critical, like real-time search and recommendation systems.
Suitable for use cases where approximate results are acceptable, such as similarity-based search in AI-driven applications.
### Exhaustive KNN:
Best for small to medium datasets or cases where absolute accuracy is required, such as scientific applications or high-precision recommendation systems.
Suitable for batch processing scenarios where speed is not a priority but accuracy is paramount.
## Summary:
HNSW offers faster and more scalable performance with approximate results, making it the go-to choice for real-time, large-scale vector search applications.
Exhaustive KNN provides exact results but is slower and less scalable, best used for small datasets or when absolute accuracy is required.

In [59]:
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)
embedding_function = embeddings.embed_query

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store the title
    SearchableField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field for filtering on document source
    SimpleField(
        name="source",
        type=SearchFieldDataType.String,
        filterable=True,
    ),
]

index_name: str = "langchain-vector-index-custom"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embedding_function,
    fields=fields,
)

### Add data and perform a query that includes a filter

This example adds data to the vector store based on the custom schema. It loads text into the title and source fields. The source field is filterable. The sample query in this section filters the results based on content in the source field.

In [60]:
# Data in the metadata dictionary with a corresponding field in the index will be added to the index.
# In this example, the metadata dictionary contains a title, a source, and a random field.
# The title and the source are added to the index as separate fields, but the random value is ignored because it's not defined in the schema.
# The random field is only stored in the metadata field.
vector_store.add_texts(
    ["Test 1", "Test 2", "Test 3"],
    [
        {"title": "Title 1", "source": "A", "random": "10290"},
        {"title": "Title 2", "source": "A", "random": "48392"},
        {"title": "Title 3", "source": "B", "random": "32893"},
    ],
)

['ZDI4ZmIzNzktODIwZC00Yjc0LWJjMmYtN2JlM2JlN2MzOTFh',
 'MDZlOWM5NmItMzFiYy00NmI2LWFhMzktYTMzMDA1MjYxNGVl',
 'NmM0MjdiZjYtZDNiMi00MGUyLWI2NjUtZjVkOGNhYmI3MDFi']

In [61]:
res = vector_store.similarity_search(query="Test 3 source1", k=3, search_type="hybrid")
res

[Document(metadata={'id': 'NmM0MjdiZjYtZDNiMi00MGUyLWI2NjUtZjVkOGNhYmI3MDFi', 'title': 'Title 3', 'source': 'B', 'random': '32893'}, page_content='Test 3'),
 Document(metadata={'id': 'ZDI4ZmIzNzktODIwZC00Yjc0LWJjMmYtN2JlM2JlN2MzOTFh', 'title': 'Title 1', 'source': 'A', 'random': '10290'}, page_content='Test 1'),
 Document(metadata={'id': 'MDZlOWM5NmItMzFiYy00NmI2LWFhMzktYTMzMDA1MjYxNGVl', 'title': 'Title 2', 'source': 'A', 'random': '48392'}, page_content='Test 2')]

In [62]:
res = vector_store.similarity_search(
    query="Test 3 source1", k=3, search_type="hybrid", filters="source eq 'A'"
)
res

[Document(metadata={'id': 'ZDI4ZmIzNzktODIwZC00Yjc0LWJjMmYtN2JlM2JlN2MzOTFh', 'title': 'Title 1', 'source': 'A', 'random': '10290'}, page_content='Test 1'),
 Document(metadata={'id': 'MDZlOWM5NmItMzFiYy00NmI2LWFhMzktYTMzMDA1MjYxNGVl', 'title': 'Title 2', 'source': 'A', 'random': '48392'}, page_content='Test 2')]

### Create a new index with a scoring profile

Here's another custom schema that includes a scoring profile definition. A scoring profile is used for relevance tuning of nonvector content, which is helpful in hybrid search scenarios.

In [63]:
from azure.search.documents.indexes.models import (
    FreshnessScoringFunction,
    FreshnessScoringParameters,
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

#  Azure OpenAI is your provider.
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)
embedding_function = embeddings.embed_query

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store the title
    SearchableField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field for filtering on document source
    SimpleField(
        name="source",
        type=SearchFieldDataType.String,
        filterable=True,
    ),
    # Additional data field for last doc update
    SimpleField(
        name="last_update",
        type=SearchFieldDataType.DateTimeOffset,
        searchable=True,
        filterable=True,
    ),
]
# Adding a custom scoring profile with a freshness function
sc_name = "custom_scoring_profile"
sc = ScoringProfile(
    name=sc_name,
    text_weights=TextWeights(weights={"title": 5}),
    function_aggregation="sum",
    functions=[
        FreshnessScoringFunction(
            field_name="last_update",
            boost=100,
            parameters=FreshnessScoringParameters(boosting_duration="P2D"),
            interpolation="linear",
        )
    ],
)

index_name = "langchain-vector-custom-scoring-profile"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    fields=fields,
    scoring_profiles=[sc],
    default_scoring_profile=sc_name,
)

In [64]:
# Adding same data with different last_update to show Scoring Profile effect
from datetime import datetime, timedelta

today = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S-00:00")
yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%S-00:00")
one_month_ago = (datetime.utcnow() - timedelta(days=30)).strftime(
    "%Y-%m-%dT%H:%M:%S-00:00"
)

vector_store.add_texts(
    ["Test 1", "Test 1", "Test 1"],
    [
        {
            "title": "Title 1",
            "source": "source1",
            "random": "10290",
            "last_update": today,
        },
        {
            "title": "Title 1",
            "source": "source1",
            "random": "48392",
            "last_update": yesterday,
        },
        {
            "title": "Title 1",
            "source": "source1",
            "random": "32893",
            "last_update": one_month_ago,
        },
    ],
)

C:\Users\vmunagal\AppData\Local\Temp\ipykernel_35132\433022458.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S-00:00")
C:\Users\vmunagal\AppData\Local\Temp\ipykernel_35132\433022458.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  yesterday = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%S-00:00")
C:\Users\vmunagal\AppData\Local\Temp\ipykernel_35132\433022458.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  one_month_ago = (datetime.ut

['ZTg2MGY1M2YtZThmNC00ODYzLTlkYWYtNzEzMDg4NWU4OGI4',
 'YjNjOGY3YmMtOTcxMi00ZTk2LWIzMTAtYzg5NDQ0ZTdiYTdl',
 'MWFiN2VjMTktMWQ4My00NWQ4LWFkMjktNmQ0ZDdhOWFmMGRi']

In [65]:
res = vector_store.similarity_search(query="Test 1", k=3, search_type="similarity")
res

[Document(metadata={'id': 'ZTg2MGY1M2YtZThmNC00ODYzLTlkYWYtNzEzMDg4NWU4OGI4', 'title': 'Title 1', 'source': 'source1', 'random': '10290', 'last_update': '2024-10-13T17:30:21-00:00'}, page_content='Test 1'),
 Document(metadata={'id': 'YjNjOGY3YmMtOTcxMi00ZTk2LWIzMTAtYzg5NDQ0ZTdiYTdl', 'title': 'Title 1', 'source': 'source1', 'random': '48392', 'last_update': '2024-10-12T17:30:21-00:00'}, page_content='Test 1'),
 Document(metadata={'id': 'MWFiN2VjMTktMWQ4My00NWQ4LWFkMjktNmQ0ZDdhOWFmMGRi', 'title': 'Title 1', 'source': 'source1', 'random': '32893', 'last_update': '2024-09-13T17:30:21-00:00'}, page_content='Test 1')]